In [0]:
display(spark.sql("SHOW CATALOGS"))

catalog
my_catalog
samples
system
workspace


In [0]:
%skip
%sql
DROP VOLUME IF EXISTS my_catalog.bronze_retailx.customers;
DROP VOLUME IF EXISTS my_catalog.bronze_retailx.orders;

DROP Table IF EXISTS my_catalog.bronze_retailx.customers;
DROP Table IF EXISTS my_catalog.bronze_retailx.orders;
DROP SCHEMA IF EXISTS my_catalog.bronze_retailx;

DROP Table IF EXISTS my_catalog.silver_retailx.customers;
DROP Table IF EXISTS my_catalog.silver_retailx.orders;
DROP VOLUME IF EXISTS my_catalog.silver_retailx.customers;
DROP VOLUME IF EXISTS my_catalog.silver_retailx.orders;
DROP SCHEMA IF EXISTS my_catalog.silver_retailx;

DROP Table IF EXISTS my_catalog.gold_retailx.customer_metrics;
DROP Table IF EXISTS my_catalog.gold_retailx.daily_sales;
DROP Table IF EXISTS my_catalog.gold_retailx.monthly_sales;
DROP Table IF EXISTS my_catalog.gold_retailx.revenue_trend;
DROP Table IF EXISTS my_catalog.gold_retailx.top_customers;

DROP VOLUME IF EXISTS my_catalog.gold_retailx.customers;
DROP VOLUME IF EXISTS my_catalog.gold_retailx.orders;
DROP SCHEMA IF EXISTS my_catalog.gold_retailx;

--DROP SCHEMA IF EXISTS my_catalog.raw_retailx;

DROP VOLUME IF EXISTS my_catalog._system.checkpoint;
DROP SCHEMA IF EXISTS my_catalog._system;

-- If not empty. 
-- SHOW VOLUMES IN my_catalog._system;

In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS my_catalog.bronze_retailx;
CREATE SCHEMA IF NOT EXISTS my_catalog.silver_retailx;
CREATE SCHEMA IF NOT EXISTS my_catalog.gold_retailx;

CREATE SCHEMA IF NOT EXISTS my_catalog.raw_retailx;
CREATE SCHEMA IF NOT EXISTS my_catalog._system;

In [0]:
%sql

CREATE VOLUME IF NOT EXISTS my_catalog.bronze_retailx.customers
COMMENT 'For Customers Raw Data';

CREATE VOLUME IF NOT EXISTS my_catalog.bronze_retailx.orders
COMMENT 'For Orders Raw Data';

CREATE VOLUME IF NOT EXISTS my_catalog._system.checkpoint
COMMENT 'Checkpoint volume for RetailX Auto Loader';

In [0]:
%sql
-- createing external volumes that we use, customers and orders.

CREATE EXTERNAL VOLUME  IF NOT EXISTS my_catalog.raw_retailx.customers
LOCATION 's3://thotadhanasekhar0001/CUSTOMERS/'
COMMENT 'External Location Volume for CUSTOMERS'; 

CREATE EXTERNAL VOLUME  IF NOT EXISTS my_catalog.raw_retailx.orders
LOCATION 's3://thotadhanasekhar0001/ORDERS/'
COMMENT 'External Location Volume for '; 

In [0]:
# seperate checkpoint directory for each data.
dbutils.fs.mkdirs("/Volumes/my_catalog/_system/checkpoint/retailx_customers/") 

True

<hr>

### SILVER

In [0]:
%skip
%sql
DROP TABLE IF EXISTS my_catalog.silver_retailx.orders;
DROP TABLE IF EXISTS my_catalog.silver_retailx.customers;

In [0]:
%sql

-- Customers table
CREATE TABLE IF NOT EXISTS my_catalog.silver_retailx.customers (
    customer_id        BIGINT NOT NULL,
    name               STRING,
    email              STRING,
    created_date       DATE,

    -- SCD2 columns
    effective_from     TIMESTAMP,
    effective_to       TIMESTAMP,
    is_current         BOOLEAN,

    -- audit
    record_hash        STRING
)
USING DELTA
TBLPROPERTIES (
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
);

-- Orders table
CREATE TABLE IF NOT EXISTS my_catalog.silver_retailx.orders (
    order_id           BIGINT,
    customer_id        BIGINT,
    order_date         DATE,
    amount             DECIMAL(10,2),

    -- audit
    _ingested_at  TIMESTAMP
)
USING DELTA
TBLPROPERTIES (
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
);

In [0]:
%sql
-- Primary Key on Customers
ALTER TABLE my_catalog.silver_retailx.customers
ADD CONSTRAINT pk_customers
PRIMARY KEY (customer_id);

-- Foreign Key on Orders refers Customers
ALTER TABLE my_catalog.silver_retailx.orders
ADD CONSTRAINT fk_orders_customers
FOREIGN KEY (customer_id)
REFERENCES my_catalog.silver_retailx.customers(customer_id);

### Just to Check

In [0]:
%skip
%sql
SHOW EXTERNAL LOCATIONS;

In [0]:
%skip
%sql
LIST 's3://thotadhanasekhar0001';

In [0]:
%skip
%sql
LIST '/Volumes/my_catalog/raw_retailx/customers/';
-- LIST 's3://thotadhanasekhar0001/ORDERS/';

In [0]:
%skip
CREATE TABLE IF NOT EXISTS my_catalog.silver_retailx.customers (
    customer_id        BIGINT,
    name               STRING,
    email              STRING,
    created_date       DATE,

    -- SCD2 columns
    effective_from     TIMESTAMP,
    effective_to       TIMESTAMP,
    is_current         BOOLEAN,

    -- audit
    record_hash        STRING
)
USING DELTA;
